In [1]:
from src.fistanet.M5FISTANet import FISTANet
from src.fistanet.loader import DataSplit
from src.fistanet.solver import Solver
from os.path import join as pjoin
from torchsummary import summary
from datetime import datetime
import numpy as np
import torch

In [2]:
DATA_DIR = './data'
DATA_FILE_GEN = 'generated/BW_master_10000_2024-04-07-12-43-32.pkl'
DATA_FILE_SIGS = 'steinbrinker/testing_data_mvg_avg.npy'
DATA_FILE_BW = 'mit-bih/bw'
DATA_FILE_BPDN = 'generated/BW_alphas-BPDN_10000_2024-04-07-12-43-32.npy'
DICT_FILE_BW = 'steinbrinker/dictionary_BW_real_data.npy'
DATA_SIZE = 10000
BATCH_SIZE = 1000
TVT_SPLIT = {
    'train': 80,
    'valid': 10,
    'test': 10
}

FNET_LAYER_NO = 4
FNET_FEATURE_NO = 16
LAMBDA_SP_LOSS = 1e-2

EPOCH_NO = 1000
START_EPOCH = 0
TEST_EPOCH = 1001
LR_DEC_AFTER = 1000
LR_DEC_EVERY = 10
LOG_INTERVAL = 4
LEARNING_RATE = 5e-4

In [3]:
trn_ldr, val_ldr, tst_ldr = DataSplit(DATA_DIR, DATA_FILE_GEN, DATA_FILE_SIGS, DATA_FILE_BW, DATA_FILE_BPDN, TVT_SPLIT, BATCH_SIZE)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
Psi = np.load(pjoin(DATA_DIR, DICT_FILE_BW))
Psi = torch.from_numpy(Psi)
Psi = Psi.clone().detach().to(device=device)

In [6]:
fista_net = FISTANet(FNET_LAYER_NO, FNET_FEATURE_NO)
fista_net = fista_net.to(device)# define arguments of fista_net


In [7]:
# summary(fista_net, input_size=(1, 64, 298), device=str(device))
print('Total number of parameters fista net:',
          sum(p.numel() for p in fista_net.parameters()))

Total number of parameters fista net: 18871


In [8]:
dt = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
args = {
    'model_name': 'FISTANet',
    'num_epochs': EPOCH_NO,
    'lr': LEARNING_RATE,
    'data_dir': DATA_DIR,
    'save_path': f'./runs/{dt}',
    'start_epoch': START_EPOCH,
    'multi_gpu': False,
    'device': device,
    'log_interval': LOG_INTERVAL,
    'test_epoch': TEST_EPOCH,
    'lr_dec_after': LR_DEC_AFTER,
    'lr_dec_every': LR_DEC_EVERY,
    'lambda_sp_loss': LAMBDA_SP_LOSS
}

In [9]:
solver = Solver(fista_net, Psi, trn_ldr, val_ldr, BATCH_SIZE, args, tst_ldr)

In [ ]:
solver.train()

Training epoch 1...

Train Epoch: 1 [0/8000 (0%)]	Batch Loss: 3834810.501187	Learning Rate (w_theta): 0.000500	 TIME:2.5s
				Disc: 3834810.314863		Spars: 0.186324
	 TVw: -0.499000 | TVb: -1.999000 | GSw: -0.201000 | GSb: 0.099000 | TSUw: 0.499000 | TSUb: 0.001000

Train Epoch: 1 [4000/8000 (50%)]	Batch Loss: 3710654.421245	Learning Rate (w_theta): 0.000500	 TIME:4.0s
				Disc: 3710654.258343		Spars: 0.162902
	 TVw: -0.496085 | TVb: -1.996202 | GSw: -0.205002 | GSb: 0.094998 | TSUw: 0.494998 | TSUb: 0.005002
Validating epoch 1...
-------------------------------------------
Epoch statistics:
Average training loss: 4170957.5297497045
Average validation loss: 3226582.7631571935
Training epoch 2...

Train Epoch: 2 [0/8000 (0%)]	Batch Loss: 3723616.561829	Learning Rate (w_theta): 0.000500	 TIME:5.9s
				Disc: 3723616.375653		Spars: 0.186176
	 TVw: -0.497391 | TVb: -1.998111 | GSw: -0.208972 | GSb: 0.091029 | TSUw: 0.491026 | TSUb: 0.008969

Train Epoch: 2 [4000/8000 (50%)]	Batch Loss: 38912

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\galiger.gergo\AppData\Local\Temp\ipykernel_73536\3719546261.py", line 1, in <module>
    solver.train()
  File "C:\Users\galiger.gergo\Desktop\ecg-denoising\workspace\src\fistanet\solver.py", line 263, in train
    self.train_losses.append(loss.item())
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\ultratb.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\galiger.gergo\AppData\Local\Temp\ipykernel_73536\3719546261.py", line 1, in <module>
    solver.train()
  File "C:\Users\galiger.gergo\Desktop\ecg-denoising\workspace\src\fistanet\solver.py", line 263, in train
    self.train_losses.append(loss.item())
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interact